In [12]:
def editDistance(x, y):
    # Create distance matrix
    D = []
    for i in range(len(x)+1):
        D.append([0]*(len(y)+1))
    # Initialize first row and column of matrix
    for i in range(len(x)+1):
        D[i][0] = i
    for i in range(len(y)+1):
        D[0][i] = 0
    # Fill in the rest of the matrix
    for i in range(1, len(x)+1):
        for j in range(1, len(y)+1):
            distHor = D[i][j-1] + 1
            distVer = D[i-1][j] + 1
            if x[i-1] == y[j-1]:
                distDiag = D[i-1][j-1]
            else:
                distDiag = D[i-1][j-1] + 1
            D[i][j] = min(distHor, distVer, distDiag)
    # Edit distance is the value in the bottom right corner of the matrix
    return min(D[-1])

In [13]:
P="GCGTATGC"
T ="TATTGGCTATACGGTT"
print(editDistance(P,T))

2


In [14]:
def readGenome(filename):
    genome = ''
    with open(filename, 'r') as f:
        for line in f:
            # ignore header line with genome information
            if not line[0] == '>':
                genome += line.rstrip()
    return genome

In [15]:
genome = readGenome('chr1.GRCh38.excerpt.fasta')

In [16]:
P ="GCTGATCGATCGTACG"
print(editDistance(P,genome))

3


In [17]:
P="GATTTACCAGATTGAG"
print(editDistance(P,genome))

2


In [116]:
from collections import defaultdict

def create_kmers_from_reads(reads, k):
        kmer_read = defaultdict(set)
        for read in reads:
            read_length = len(read)
            kmers = [read[i:i+k] for i in range(read_length - k + 1)]
            for kmer in kmers:
                kmer_read[kmer].add(read)
        return kmer_read


In [117]:
t = "GATTA"
k = 3
create_kmers_from_reads([t], k)

defaultdict(set, {'GAT': {'GATTA'}, 'ATT': {'GATTA'}, 'TTA': {'GATTA'}})

In [23]:
#!wget https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/ERR266411_1.for_asm.fastq

--2021-02-20 15:02:27--  https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/ERR266411_1.for_asm.fastq
Resolving d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)... 143.204.148.139, 143.204.148.89, 143.204.148.164, ...
Connecting to d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)|143.204.148.139|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2562951 (2.4M) [application/octet-stream]
Saving to: ‘ERR266411_1.for_asm.fastq’

ERR266411_1.for_asm 100%[===================>]   2.44M  5.21MB/s    in 0.5s    

2021-02-20 15:02:29 (5.21 MB/s) - ‘ERR266411_1.for_asm.fastq’ saved [2562951/2562951]



In [24]:
def readFastq(filename):
    sequences = []
    qualities = []
    with open(filename) as fh:
        while True:
            fh.readline()  # skip name line
            seq = fh.readline().rstrip()  # read base sequence
            fh.readline()  # skip placeholder line
            qual = fh.readline().rstrip() # base quality line
            if len(seq) == 0:
                break
            sequences.append(seq)
            qualities.append(qual)
    return sequences, qualities

In [26]:
sequences, _ = readFastq("ERR266411_1.for_asm.fastq")

In [27]:
len(sequences)

10000

In [29]:
tiny_sequences = sequences[:10]

In [30]:
tiny_sequences

['TAAACAAGCAGTAGTAATTCCTGCTTTATCAAGATAATTTTTCGACTCATCAGAAATATCCGAAAGTGTTAACTTCTGCGTCATGGAAGCGATAAAACTC',
 'AACAAGCAGTAGTAATTCCTGCTTTATCAAGATAATTTTTCGACTCATCAGAAATATACGAAAGTGTTAACTTCTGCGTCATGGACACGAAAAAACTCCC',
 'AACAAGCAGTAGTAATTCCTGCTTTATCAAGATAATTTTTCGACTCATCAGAAATATCCGAAAGTGTTAACTTCTGCGTCATGGAAGCGATAAAACTCTG',
 'AGCCGACGTTTTGGCGGCGCAACCTGTGACGACAAATCTGCTCAAATTTATGCGCGCTTCGATAAAAATGATTGGCGTATCCAACCTGCAGAGTTTTATC',
 'GACAAATCTGCTCAAATTTATGCGCGCTTCGATAAAAATGATTGGCGTATCCAACCTGCAGAGTTTTATCGCTTCCATGACGCAGAAGTTAACACTTTCG',
 'CTGTAGCCGACGTTTTGGCGGCGCAACCTGTGACGACAAATCTGCTCAAATTTATGCGCGCTTCGATAAAAATGATTGGCGTATCCAACCTGCAGAGTTT',
 'CTGTGACGACAAATCTGCTCAAATTTATGCGCGCTTCGATAAAAATGATTGGCGTATCCAACCTGCAGAGTTTTATCGCTTCCATGACGCAGAAGTTAAC',
 'CAAATCTGCTCAAATTTATGCGCGCTTCGATAAAAATGATTGGCGTATCCAACCTGCAGAGTTTTATCGCTTCCATGACGCAGAAGTTAACACTTTCGGA',
 'GTAAACAAGCAGTAGTAATTCCTGCTTTATCAAGATAATTTTTCGACTCATCAGCAATATCCGAAAGAGTTAACTTTTGCGTCATGGAAGCGATAAAACC',
 'GTAAACAAGCAGTAGTAATTCCTGCTTTATCAAGATAATTTTTCGACTCATCA

In [60]:
#d = create_kmers_from_reads(tiny_sequences, 30)

In [118]:
def overlap(a, b, min_length=3):
    """ Return length of longest suffix of 'a' matching
        a prefix of 'b' that is at least 'min_length'
        characters long.  If no such overlap exists,
        return 0. """
    start = 0  # start all the way at the left
    while True:
        start = a.find(b[:min_length], start)  # look for b's prefix in a
        if start == -1:  # no more occurrences to right
            return 0
        # found occurrence; check for full suffix/prefix match
        if b.startswith(a[start:]):
            return len(a)-start
        start += 1  # move just past previous match

In [119]:
def overlap_all_pairs(reads, min_length = 30):

    #create dict with kmer key/and set of reads with that kmer value. 
    kmer_dict = create_kmers_from_reads(reads, min_length) 
        
    overlap_graph = defaultdict(set) #read is key, set of overlapping reads are value
    
    
    for read in reads:
        #create suffix for this read
        read_suffix = read[-min_length: ] 
        
        #extract set of all reads containing this kmer/suffix
        read_set = kmer_dict[read_suffix]
        
        assert(len(read_set) > 0) # check that the set isnt empty
        
        read_set.remove(read) #remove the read so we dont compare it with itself
        
        for compar_read in read_set:
            if overlap(read, compar_read, min_length):
                overlap_graph[read].add(compar_read)
    return overlap_graph

In [120]:
def count_elements(d):
    count_edges = 0
    count_nodes = 0
    for read, read_set in d.items():
        if read_set:
            count_edges += len(read_set)
            count_nodes += 1
    return count_nodes,count_edges 

In [121]:
reads = ['ABCDEFG', 'EFGHIJ', 'HIJABC']
d = overlap_all_pairs(reads, 3)

In [122]:
count_nodes, count_edges = count_elements(d)
print(count_nodes, count_edges)

3 3


In [131]:
overlap_graph = overlap_all_pairs(tiny_sequences)

In [132]:
count_nodes, count_edges = count_elements(overlap_graph)
print(count_nodes, count_edges)

6 13


In [133]:
overlap_graph = overlap_all_pairs(sequences)
count_nodes, count_edges = count_elements(overlap_graph)
print(count_nodes, count_edges)

7161 904746
